In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Configuration
DATA_ROOTS = [
    r"/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR",
    r"/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian"
]
BATCH_SIZE = 32
NUM_WORKERS = 4  # Optimize based on CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class
class ParkinsonSpectrogramDataset(Dataset):
    def __init__(self, root_dirs, transform=None):
        if isinstance(root_dirs, str):  # If a single path is given, convert it to a list
            root_dirs = [root_dirs]
        self.root_dirs = root_dirs
        self.transform = transform
        self.samples = self._load_samples()
        
    def _load_samples(self):
        samples = []
        for root_dir in self.root_dirs:
            for class_name in ['HC', 'PD']:
                class_dir = os.path.join(root_dir, class_name)
                if not os.path.exists(class_dir):
                    continue

                # Traverse subfolders inside HC/PD
                for patient_folder in os.listdir(class_dir):
                    patient_path = os.path.join(class_dir, patient_folder)
                    if os.path.isdir(patient_path):  # Ensure it's a directory
                        for img_file in os.listdir(patient_path):
                            if img_file.lower().endswith('.png'):  # Only PNG images
                                img_path = os.path.join(patient_path, img_file)
                                samples.append((img_path, 0 if class_name == 'HC' else 1))

        print(f"✅ Loaded {len(samples)} samples from {self.root_dirs}")
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            img = self.transform(img)
        return img, label

# Data Transforms
train_transform = transforms.Compose([
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Only slight translations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between -1 and 1
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Create Datasets (Loading from both MDVR & Italian datasets)
train_dataset = ParkinsonSpectrogramDataset([os.path.join(root, 'train') for root in DATA_ROOTS], transform=train_transform)
val_dataset = ParkinsonSpectrogramDataset([os.path.join(root, 'val') for root in DATA_ROOTS], transform=test_transform)
test_dataset = ParkinsonSpectrogramDataset([os.path.join(root, 'test') for root in DATA_ROOTS], transform=test_transform)

# Optimized DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=NUM_WORKERS, pin_memory=True)

print(f"✅ DataLoaders ready! Using device: {device}")

✅ Loaded 154732 samples from ['/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR/train', '/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian/train']
✅ Loaded 37807 samples from ['/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR/val', '/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian/val']
✅ Loaded 41886 samples from ['/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR/test', '/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian/test']
✅ DataLoaders ready! Using device: cuda


In [5]:
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)
print(torch.version.cuda)

True
True
12.6


In [6]:
print(torch.cuda.get_device_capability(device))
# Should return (7, 5) or higher for good AMP support

(7, 5)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.amp import GradScaler, autocast
from tqdm import tqdm
import time
import numpy as np
from sklearn.metrics import classification_report

# ==================
# Model Definition
# ==================

class DenseNet121(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Load with weights=None and disable unnecessary features
        self.densenet = models.densenet121(weights=None, progress=False)
        
        # Optimized conv0 modification
        self.densenet.features.conv0 = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        # Initialize with better memory alignment
        nn.init.kaiming_normal_(self.densenet.features.conv0.weight, mode='fan_out')
        
        # Replace classifier with more efficient version
        in_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        with torch.amp.autocast(device_type = 'cuda'):  # Built-in autocast
            return self.densenet(x)


# ==================
# Training Setup
# ==================

def create_model(num_classes, device):
    model = DenseNet121(num_classes).to(device)
    
    # Weight initialization
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
    
    return model

# ==================
# Training Loop
# ==================

def train_model(model, train_loader, val_loader, num_classes, device, 
                input_size=(224, 224), mean=0.5, std=0.5, num_epochs=30):

    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    # Hyperparameters
    optimizer = optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()
    best_acc = 0.0
    
    # Time tracking
    total_start = time.time()
    epoch_times = []
    
    for epoch in range(num_epochs):  # Adjust epochs as needed
        epoch_start = time.time()
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        # Progress bar setup
        batch_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}', 
                         leave=False, dynamic_ncols=True)
        
        for batch_idx, (inputs, labels) in enumerate(batch_pbar):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # On-the-fly preprocessing
            inputs = torch.nn.functional.interpolate(inputs, size=input_size, 
                                                    mode='bicubic', 
                                                    align_corners=False)
            inputs = (inputs - mean) / std  # Normalize
            
            # Mixed precision training
            with autocast(device_type = 'cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Backpropagation
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
            # Metrics
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            batch_pbar.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Acc': f"{100.*correct/total:.2f}%",
                'GPU': f"{torch.cuda.memory_allocated()/1e9:.2f}GB"
            })
        
        # Validation
        val_metrics = evaluate_model(model, val_loader, num_classes, device, 
                                    input_size, mean, std)
        
        # Epoch timing
        epoch_time = time.time() - epoch_start
        epoch_times.append(epoch_time)
        avg_time = np.mean(epoch_times)
        remaining = avg_time * (50 - epoch - 1)
        
        # Progress update
        print(f"\nEpoch {epoch+1:02d} Summary:")
        print(f"Train Loss: {train_loss/(batch_idx+1):.4f} | Acc: {100.*correct/total:.2f}%")
        print(f"Val Loss: {val_metrics['loss']:.4f} | Acc: {100.*val_metrics['accuracy']:.2f}%")
        print(f"Time: {epoch_time:.1f}s | Total: {time.time()-total_start:.1f}s | "
              f"ETA: {remaining:.1f}s\n")
        
        # Save best model
        if val_metrics['accuracy'] > best_acc:
            best_acc = val_metrics['accuracy']
            torch.save(model.state_dict(), 'best_model4.pth')
        
        scheduler.step(val_metrics['accuracy'])
    
    return model

# ==================
# Evaluation
# ==================

def evaluate_model(model, loader, num_classes, device, input_size, mean, std):
    model.eval()
    loss_total = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc='Evaluating', leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Preprocessing
            inputs = torch.nn.functional.interpolate(inputs, size=input_size, 
                                                    mode='bicubic', 
                                                    align_corners=False)
            inputs = (inputs - mean) / std
            
            # Inference
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Store results
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            loss_total += nn.CrossEntropyLoss()(outputs, labels).item()
    
    # Calculate metrics
    report = classification_report(all_labels, all_preds, output_dict=True)
    return {
        'loss': loss_total/len(loader),
        'accuracy': report['accuracy'],
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'f1': report['weighted avg']['f1-score'],
        'report': report
    }

# ==================
# Main Execution
# ==================

if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.backends.cudnn.benchmark = True  # Optimize for fixed input size
    
    # User configuration
    num_classes = 10  # Set according to your dataset
    input_size = (224, 224)  # Optimal size determined through experiments
    mean = 0.5  # Calculate from your dataset
    std = 0.5   # Calculate from your dataset
    
    # Initialize model
    model = create_model(num_classes, device)
    
    # Train model
    trained_model = train_model(model, train_loader, val_loader, num_classes, 
                               device, input_size, mean, std)
    
    # Final evaluation
    test_metrics = evaluate_model(trained_model, test_loader, num_classes,
                                 device, input_size, mean, std)
    
    print("\nFinal Test Results:")
    print(f"Accuracy: {100.*test_metrics['accuracy']:.2f}%")
    print(f"Precision: {test_metrics['precision']:.4f}")
    print(f"Recall: {test_metrics['recall']:.4f}")
    print(f"F1-Score: {test_metrics['f1']:.4f}")

Epoch 1:   1%| | 57/4836 [01:24<1:29:17,  1.12s/it